## Robos Pretrained Loader

Este notebook tiene propósito cargar el modelo fine-tuned (ajustado) en Google Colab y el modelo fine tuned en mi PC y realizar las preubas de testing

## Generación del Dataset de Prueba con load_dataset - tokenización - 

In [1]:
import os
import pandas as pd
dir_root = os.getcwd()

In [2]:
dataset_df = pd.read_csv(os.path.join(dir_root, 'data/interim/trainset.csv'), converters={'NDD':str})
# dataset_df.sample(5)

In [3]:
labels_set = set(dataset_df.labels.to_list())
labels_set

{'ROBO A DOMICILIO',
 'ROBO A PERSONAS',
 'ROBO A UNIDADES ECONOMICAS',
 'ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEHICULOS',
 'ROBO DE CARROS',
 'ROBO DE MOTOS'}

In [4]:
from datasets import load_dataset, Features, ClassLabel, Value
# traincsv = os.path.join(dir_root, '/content/drive/MyDrive/data/interim/trainsethugf.csv')
# traincsv = os.path.join(dir_root, 'data/interim/trainsethugf.csv')
# testcsv = os.path.join(dir_root, '/content/drive/MyDrive/data/interim/testsethugf.csv')
testcsv = os.path.join(dir_root, 'data/interim/testsethugf.csv')
# validcsv = os.path.join(dir_root, '/content/drive/MyDrive/data/interim/validsethugf.csv')
# validcsv = os.path.join(dir_root, 'data/interim/validsethugf.csv')
# class_names = ["RoboADomicilio", "RoboAPersonas", "RoboAUnidadesEconomicas", "RoboDeBienesAccesoriosYAutoPartes", "RoboDeCarros", "RoboDeMotos"]
class_names = list(labels_set)
robo_features = Features({'relato': Value('string'), 'labels': ClassLabel(names=class_names)})
dataset = load_dataset("csv", data_files={'test': testcsv}, features=robo_features)

C:\ProgramData\Anaconda3\envs\tfenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration default-dff74bfcd49b570b


Extracting data files: 100%|████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 333.17it/s]


Dataset csv downloaded and prepared to C:\Users\entea\.cache\huggingface\datasets\csv\default-dff74bfcd49b570b\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 66.68it/s]


In [5]:
dataset

DatasetDict({
    test: Dataset({
        features: ['relato', 'labels'],
        num_rows: 90000
    })
})

In [6]:
from transformers import DistilBertTokenizer, AutoTokenizer, DistilBertTokenizerFast
model_name = 'distilbert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
seqlen = dataset_df.relato.apply(lambda x: len(x.split()))
seqlen.max()

300

In [8]:
def tokenizer_func(examples):
  return tokenizer(examples["relato"],
                   max_length=seqlen.max(),
                   padding = "max_length",
                   truncation=True)

In [9]:
tokenized_dataset = dataset.map(tokenizer_func, batched=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [00:21<00:00,  4.17ba/s]


## Generación del Dataset de Prueba con tensorflow slices - tokenización - 

In [10]:
tokenizer('Artificial Intelligence is a complex science  that sometimes needs philosophy')

{'input_ids': [101, 90841, 69229, 28854, 10124, 169, 17217, 14729, 10189, 17611, 28615, 33357, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [12]:
# Generando muestra de 4000 para tf_test_set
tf_test_set = tokenized_dataset["test"].shuffle(seed=42).select(range(200)).to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [13]:
for i, batch in enumerate(tf_test_set):
    print(batch)
    if i > 3:
        break

{'labels': <tf.Tensor: shape=(16,), dtype=int64, numpy=array([0, 5, 0, 0, 5, 0, 5, 2, 2, 3, 1, 2, 0, 0, 0, 1], dtype=int64)>, 'input_ids': <tf.Tensor: shape=(16, 300), dtype=int64, numpy=
array([[  101, 10125, 10671, ...,     0,     0,     0],
       [  101, 10196, 10125, ..., 10162, 10215,   102],
       [  101, 10196, 10125, ...,     0,     0,     0],
       ...,
       [  101, 10196, 10125, ...,     0,     0,     0],
       [  101, 10196, 10125, ...,     0,     0,     0],
       [  101, 10196, 10125, ...,     0,     0,     0]], dtype=int64)>, 'attention_mask': <tf.Tensor: shape=(16, 300), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int64)>}
{'labels': <tf.Tensor: shape=(16,), dtype=int64, numpy=array([3, 5, 0, 2, 3, 0, 5, 2, 1, 2, 3, 2, 2, 2, 1, 3], dtype=int64)>, 'input_ids': <tf.Tensor: shape=(16,

## Generación del dataset con Slices

In [14]:
test_samples = tokenized_dataset["test"].shuffle(seed=42).select(range(200)) #.shuffle(seed=42)

Loading cached shuffled indices for dataset at C:\Users\entea\.cache\huggingface\datasets\csv\default-dff74bfcd49b570b\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58\cache-f8fc9d2bcda1ef73.arrow


In [15]:
test_samples['labels'][:10]

[0, 5, 0, 0, 5, 0, 5, 2, 2, 3]

Generando los feature vectors

In [16]:
# Generando los feature vectors
test_features = test_samples.remove_columns(["relato", "labels"]).with_format("tensorflow") # use tokenized_dataset to use full dataset

In [17]:
test_features = {x: test_features[x] for x in tokenizer.model_input_names}

In [18]:
test_features

{'input_ids': <tf.Tensor: shape=(200, 300), dtype=int64, numpy=
 array([[  101, 10125, 10671, ...,     0,     0,     0],
        [  101, 10196, 10125, ..., 10162, 10215,   102],
        [  101, 10196, 10125, ...,     0,     0,     0],
        ...,
        [  101, 10125, 13101, ...,     0,     0,     0],
        [  101, 47599, 43645, ...,     0,     0,     0],
        [  101, 10125, 14184, ...,     0,     0,     0]], dtype=int64)>,
 'attention_mask': <tf.Tensor: shape=(200, 300), dtype=int64, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int64)>}

Generando las etiquetas

In [19]:
from tensorflow.keras.utils import to_categorical
test_labels = to_categorical(test_samples["labels"])

In [20]:
test_labels[:10,:]

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]], dtype=float32)

## Carga del Modelo
Se prueban dos modelos. El entrenado en el computador personal y el entrenado en Colab

In [54]:
from transformers import TFAutoModelForSequenceClassification, TFDistilBertForSequenceClassification, TFAutoModel, AutoModelForSequenceClassification, AutoConfig
config = AutoConfig.from_pretrained('./models/robospretrained1000/config.json')
model = TFAutoModelForSequenceClassification.from_pretrained('./models/robospretrained1000/', local_files_only=True, config=config)
# model = TFAutoModelForSequenceClassification.from_pretrained('./models/robosweights1k/')
# model = AutoModelForSequenceClassification.from_pretrained('models/robospretrained1000/', from_tf=True, config=config)

Some layers from the model checkpoint at ./models/robospretrained1000/ were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./models/robospretrained1000/ and are newly initialized: ['dropout_119']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [86]:
# model = tf.keras.models.load_model('models/robossave/')

In [42]:
import tensorflow as tf
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
metrics=tf.keras.metrics.CategoricalAccuracy()
# metrics=tf.keras.metrics.Accuracy()
# model.compile(optimizer=optimizer, metrics = ['accuracy', 'AUC', 'Precision', 'Recall'])
model.compile(optimizer=optimizer, metrics = metrics, loss=loss)

AttributeError: 'DistilBertForSequenceClassification' object has no attribute 'compile'

In [23]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  134734080 
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  4614      
_________________________________________________________________
dropout_19 (Dropout)         multiple                  0         
Total params: 135,329,286
Trainable params: 135,329,286
Non-trainable params: 0
_________________________________________________________________


## Testeo del modelo en dataset cargado mediante huggingface load_dataset

In [24]:
import tensorflow_datasets as tfds
import numpy as np
test_npy = tfds.as_numpy(tf_test_set)

In [25]:
iterable = list(batch['labels'] for batch in test_npy)
y_true = np.hstack(iterable)

In [26]:
y_true.shape

(200,)

In [27]:
y_logits = model.predict(tf_test_set).logits
yprob = tf.nn.softmax(y_logits, axis=-1).numpy()
yprob[:10]

array([[1.98787555e-01, 5.03528165e-03, 2.58802832e-03, 3.58307501e-04,
        7.78779387e-01, 1.44513864e-02],
       [2.71832943e-01, 1.13147618e-02, 2.37117463e-04, 1.82100819e-04,
        3.92450429e-02, 6.77187979e-01],
       [4.74163936e-03, 1.41619742e-01, 1.65109605e-01, 4.80766371e-02,
        6.30666196e-01, 9.78618581e-03],
       [9.76291373e-02, 1.36636853e-01, 1.62562221e-01, 4.52137917e-01,
        1.50388047e-01, 6.45814987e-04],
       [7.43349269e-02, 8.40020999e-02, 1.00435654e-03, 3.47870984e-04,
        8.52096640e-03, 8.31789732e-01],
       [3.25737484e-02, 3.34094688e-02, 7.89692551e-02, 1.00340759e-02,
        8.16217542e-01, 2.87959483e-02],
       [1.50968373e-01, 9.06565785e-02, 1.87952723e-03, 4.73754568e-04,
        2.51263510e-02, 7.30895400e-01],
       [1.73550844e-02, 4.03932557e-02, 1.68228269e-01, 3.73745002e-02,
        7.35150874e-01, 1.49802875e-03],
       [4.82812850e-03, 1.83567926e-02, 7.68390000e-01, 1.04308344e-01,
        9.33034793e-02, 

In [28]:
yhat = tf.argmax(yprob, axis=1)

In [29]:
yhat_onehot = tf.one_hot(yhat, depth=6, dtype=tf.float32)
yhat_onehot[:20]

<tf.Tensor: shape=(20, 6), dtype=float32, numpy=
array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.]], dtype=float32)>

In [30]:
ytrue_onehot = tf.one_hot(y_true, depth=6, dtype=tf.float32)

In [31]:
from sklearn.metrics import  classification_report
print(classification_report(y_true=ytrue_onehot, y_pred=yhat_onehot))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        79
           1       0.62      0.48      0.54        27
           2       0.69      0.55      0.61        33
           3       0.00      0.00      0.00        15
           4       0.05      0.50      0.08        10
           5       0.77      0.75      0.76        36

   micro avg       0.32      0.32      0.32       200
   macro avg       0.35      0.38      0.33       200
weighted avg       0.34      0.32      0.31       200
 samples avg       0.32      0.32      0.32       200



## Testeo del modelo en datset cargado mediante tensorflow slices

In [32]:
y_true = test_labels
y_true.shape

(200, 6)

In [33]:
y_true[:10]

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]], dtype=float32)

In [34]:
# Performing inference with our model
predictions = model.predict(test_features)

In [35]:
# Converting predicted logits to probabilities
predictions = tf.nn.softmax(predictions.logits)

# Extracting the indices with the highest probabilities
predictions = tf.argmax(predictions, axis=1).numpy()

In [36]:
predictions

array([4, 5, 4, 3, 5, 4, 5, 4, 2, 4, 2, 2, 4, 4, 4, 1, 5, 5, 4, 4, 5, 4,
       1, 1, 1, 2, 4, 2, 2, 3, 1, 5, 4, 4, 4, 4, 0, 4, 2, 5, 4, 2, 2, 4,
       4, 2, 4, 4, 4, 4, 4, 4, 1, 5, 5, 4, 4, 4, 4, 5, 5, 4, 1, 1, 4, 4,
       4, 1, 1, 4, 1, 5, 4, 2, 4, 0, 0, 4, 2, 5, 5, 5, 4, 4, 4, 4, 4, 1,
       1, 4, 4, 2, 4, 4, 1, 4, 4, 1, 4, 4, 4, 4, 4, 4, 2, 5, 4, 2, 4, 4,
       4, 4, 4, 4, 4, 4, 3, 2, 5, 2, 4, 5, 0, 4, 4, 4, 2, 1, 1, 4, 4, 4,
       4, 5, 4, 2, 1, 4, 2, 5, 2, 5, 4, 4, 4, 1, 4, 2, 5, 5, 2, 5, 4, 5,
       4, 5, 4, 4, 5, 4, 5, 4, 5, 4, 2, 4, 1, 4, 4, 4, 4, 5, 4, 4, 1, 4,
       4, 4, 2, 5, 4, 4, 4, 5, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 2, 4, 5,
       4, 4], dtype=int64)

In [37]:
predictions_onehot = tf.one_hot(predictions, depth=6, dtype=tf.int32)
predictions_onehot.shape

TensorShape([200, 6])

In [38]:
predictions_onehot[:10,:]

<tf.Tensor: shape=(10, 6), dtype=int32, numpy=
array([[0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0]])>

In [39]:
from tensorflow.keras.utils import to_categorical
to_categorical(predictions[:10])

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]], dtype=float32)

In [40]:
print(classification_report(y_true=y_true, y_pred=predictions_onehot))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        79
           1       0.62      0.48      0.54        27
           2       0.69      0.55      0.61        33
           3       0.00      0.00      0.00        15
           4       0.05      0.50      0.08        10
           5       0.77      0.75      0.76        36

   micro avg       0.32      0.32      0.32       200
   macro avg       0.35      0.38      0.33       200
weighted avg       0.34      0.32      0.31       200
 samples avg       0.32      0.32      0.32       200



In [124]:
# model2.load_weights('models/robosweights1k/')

In [126]:
# model2.predict(test_features)